# Multi-Agent Security Mesh Demo

This notebook demonstrates the DDoS Sentinel Agent's multi-agent security mesh for detecting coordinated attacks across network, DNS, and supply chain domains.

**Author**: Oluwafemi Idiakhoa  
**Version**: 0.2.0  
**Framework**: SafeDeepAgent

## Setup

Import the multi-agent mesh components from the clean API:

In [ ]:
from ddos_sentinel import (
    SecurityMeshOrchestrator,
    DDoSSentinelAgent,
    DNSIntegrityAgent,
    DNSObservation,
    SupplyChainGuardianAgent,
    SupplyChainObservation,
    TrafficSimulator
)
from safedeepagent.core.safe_agent import SafeDeepAgent, SafeConfig

print("✓ Imports successful")

## Initialize Security Mesh

Create the multi-agent mesh with 3 specialized domain agents:

In [ ]:
# Initialize SafeDeepAgent for orchestration
safe_agent = SafeDeepAgent(safe_config=SafeConfig())

# Initialize domain-specific agents
network_agent = DDoSSentinelAgent(sensitivity=0.8)
dns_agent = DNSIntegrityAgent(sensitivity=0.8)
supply_chain_agent = SupplyChainGuardianAgent(sensitivity=0.8)

# Create mesh orchestrator
mesh = SecurityMeshOrchestrator(
    agents=[network_agent, dns_agent, supply_chain_agent],
    safe_agent=safe_agent
)

print(f"✓ Security mesh initialized with {len(mesh.agents)} agents")
print(f"  Domains: {mesh.list_domains()}")

## Scenario 1: Clean Baseline

Simulate normal operations across all domains:

In [ ]:
simulator = TrafficSimulator(seed=42)

# Generate normal observations
observations_clean = {
    'network': simulator.generate_normal_traffic(duration_seconds=30, base_pps=1000),
    'dns': DNSObservation(
        domain="example.com",
        qps=50.0,
        unique_client_ips=30,
        asn_distribution={"AS1234": 15, "AS5678": 10, "AS9012": 5},
        query_types={"A": 25, "AAAA": 20, "MX": 5},
        http_traffic_ratio=0.8
    ),
    'supply_chain': SupplyChainObservation(
        release_id="v2.3.0",
        version="2.3.0",
        signing_key_id="KEY_PROD_001",
        build_host="build-server-01.company.com",
        rollout_speed=800.0,
        total_devices_updated=5000,
        deployment_duration_hours=6.0,
        post_release_traffic_multiplier=1.1,
        is_known_signing_key=True,
        build_host_reputation="trusted",
        device_behavior_anomalies=2
    )
}

# Run analysis
result_clean = mesh.run_end_to_end(observations_clean)

print(f"Attacks detected: {result_clean['summary']['attacks_detected']}")
print(f"Global severity: {result_clean['summary']['global_severity']}")

## Scenario 2: Multi-Vector Attack (Aisuru-style)

Simulate a coordinated attack across all domains:

In [ ]:
simulator.clear_buffer()

# Generate attack observations
observations_attack = {
    'network': simulator.generate_aisuru_ddos_traffic(
        duration_seconds=30,
        attack_pps=180000,
        botnet_size=6000
    ),
    'dns': DNSObservation(
        domain="cloudflare-rank-target.com",
        qps=7500.0,
        unique_client_ips=3200,
        asn_distribution={"AS6666": 2300, "AS7777": 650, "AS8888": 250},
        query_types={"A": 7000, "AAAA": 400, "MX": 100},
        http_traffic_ratio=0.06  # Very low = fake queries
    ),
    'supply_chain': SupplyChainObservation(
        release_id="v3.0.0_MALICIOUS",
        version="3.0.0",
        signing_key_id="KEY_COMPROMISED_666",
        build_host="attacker-build-server.evil.net",
        rollout_speed=32000.0,
        total_devices_updated=55000,
        deployment_duration_hours=1.3,
        post_release_traffic_multiplier=18.0,
        is_known_signing_key=False,
        build_host_reputation="suspicious",
        device_behavior_anomalies=15000
    )
}

# Run analysis
result_attack = mesh.run_end_to_end(observations_attack)

print(f"Attacks detected: {result_attack['summary']['attacks_detected']}")
print(f"Global severity: {result_attack['summary']['global_severity']}")
print(f"Affected domains: {result_attack['summary']['affected_domains']}")

## View Per-Agent Analysis Results

In [ ]:
for analysis in result_attack['per_agent_analyses']:
    print(f"\n{'='*70}")
    print(f"Domain: {analysis.domain.upper()}")
    print(f"Attack Detected: {analysis.attack_detected}")
    if analysis.attack_detected:
        print(f"Severity: {analysis.severity.value.upper()}")
        print(f"Confidence: {analysis.confidence:.0%}")
        print(f"Indicators: {len(analysis.indicators)}")
        print(f"Notes: {analysis.notes[:200]}...")

## View Global Mitigation Plan

In [ ]:
global_plan = result_attack['global_plan']

print(f"Global Severity: {global_plan.severity.value.upper()}")
print(f"Response Time: {global_plan.recommended_response_time}")
print(f"Total Actions: {global_plan.action_count()}")

print("\n--- IMMEDIATE ACTIONS ---")
for i, action in enumerate(global_plan.immediate_actions[:5], 1):
    print(f"{i}. {action.description}")
    print(f"   Target: {action.target}, Priority: {action.priority}")

print(f"\n--- FOLLOW-UP ACTIONS ---")
print(f"{len(global_plan.follow_up_actions)} follow-up actions planned")

## Sample Output for Article

This is the kind of output you'll see in production:

In [ ]:
sample_output = """
[NETWORK] CRITICAL: Detected 180k PPS UDP flood with 6000 unique IPs.
[DNS] HIGH: Sudden spike in QPS (7500), domain popularity manipulation via resolver spam.
[SUPPLY_CHAIN] CRITICAL: Unsigned firmware v3.0.0 from suspicious host distributed to 55k devices.
[MESH] CRITICAL: Multi-vector coordinated attack detected.
[MESH] Attack Pattern: DDoS + DNS abuse + Supply-chain compromise (Aisuru-style)
[MESH] Recommended actions:
 - Rate limit UDP traffic > 50k PPS
 - Sinkhole suspicious domains
 - Roll back firmware v3.0.0
 - Notify security operations
"""

print(sample_output)

## Summary

This notebook demonstrated:

1. **Clean API** - Simple imports from `ddos_sentinel`
2. **Multi-Agent Architecture** - 3 specialized domain agents
3. **Cross-Domain Correlation** - Detecting coordinated attacks
4. **SafeDeepAgent Integration** - Security at agent and orchestrator levels
5. **Unified Mitigation Planning** - Global response across all domains

For more details, see:
- [GitHub Repository](https://github.com/oluwafemidiakhoa/DDoSSentinelAgent)
- [Internal Design Docs](../docs/INTERNAL_DESIGN.md)
- [SafeDeepAgent Framework](https://github.com/oluwafemidiakhoa/Deepagent)